In [2]:
import gymnasium as gym
from gymnasium.utils import play
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import load_model
import keras
import numpy as np

from keras import backend 

In [3]:
env = gym.make('ALE/Boxing-v5', render_mode = 'human', obs_type = 'grayscale')
#or for grayscale
#env = gym.make

In [14]:
model = load_model('models\model_cnn_stack.h5')


In [28]:
def crop_image(image):
    return image[30:180, 30:130]

def first_stack(image):
    return np.stack((image, image, image, image,image), axis=2)

def stack_image(image, stack):
    #inputs (150,100) and (1, 150, 100, 5)
    new_stack = np.zeros((1,150,100,5))
    new_stack[0,:,:,0:4] = stack[0,:,:,1:5]
    new_stack[0,:,:,4] = image
    return new_stack



def reshape_image(image):
    return np.expand_dims(image, 0)

In [8]:
def gen_labels(env):
    labels = np.zeros(18)
    for k in range(18):
        old_state = env.unwrapped.clone_state(include_rng=False)
        labels[k] = env.step(k)[1]
        env.unwrapped.restore_state(old_state)
    return tf.stack(labels.reshape((1, 18)))

In [29]:
# Now we need to train the model
# We will generate the vector of labels with the environment.step() function
# We will fit the model using those labels and the images
# We will use the model to predict the next action and then use that action to generate the next image and label
# We will repeat this process until the model is trained

# We will use the epsilon-greedy algorithm to generate the labels

epochs = 3
epsilon = 0.1
batch_size = 1

cropped_image = crop_image(env.reset()[0])
img = tf.stack(first_stack(cropped_image).reshape((1, 150, 100, 5)))

for i in range(epochs):
    cropped_image = crop_image(env.reset()[0])
    img = tf.stack(first_stack(cropped_image).reshape((1, 150, 100, 5)))
    for j in range(1000):
        # Generate the labels
        labels = gen_labels(env)

        # Fit the model

        # Predict the next action
        out = model.predict(img)
        action = np.argmax(out)

        # Epsilon-greedy
        if np.random.rand() < epsilon:
            action = env.action_space.sample()

        # Generate the next image and label
        cropped_image = crop_image(env.step(action)[0])
        print(cropped_image.shape, img.shape)
        img = stack_image(cropped_image, img)
        img = tf.stack(img.reshape((1, 150, 100, 5)))

        # Check if the game is over
        if env.unwrapped.ale.game_over():
            break
    print('Epoch', i, 'finished')
    env.reset()



1/1 [==============================] - 0s 43ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 86ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 36ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 36ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 34ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 35ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 42ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 34ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 37ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 35ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 37ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0s 34ms/step
(150, 100) (1, 150, 100, 5)
1/1 [==============================] - 0

KeyboardInterrupt: 

In [59]:
a = env.reset()
img = crop_image(a[0])
img = first_stack(img)
for i in range(1000):
    env.render()
    img = crop_image(a[0])
    stack = stack_image(img, stack)
    a = env.step(np.argmax(model.predict(tf.stack(stack.reshape((1, 150, 100, 5))))[0]))
    env.render()


1/1 [==============================] - 0s 27ms/step


In [ ]:
#save the model
model.save('models\model_cnn_stack_epsilon_greedy.h5')